In [2]:
!pip install sentence-transformers groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.1 MB/s eta 0:00:00


In [3]:
!pip install datasets
!pip install transformers faiss-cpu torch datasets
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 52.6 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.7 MB/s eta 0:00:00


### Load the dataset- HotspotQA

In [4]:
from datasets import load_dataset

# Load the HotpotQA dataset
dataset = load_dataset("hotpot_qa", "fullwiki")

# Check out the dataset structure
print(dataset)

hotpot_qa.py:   0%|          | 0.00/6.42k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.19k [00:00<?, ?B/s]

The repository for hotpot_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hotpot_qa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 90447
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
})


#### Taking only 100 data points due to computational constraints

In [5]:
from tqdm import tqdm
context = []
for i in tqdm(dataset['train']['context'][:100]):
    for j in i['sentences']:
        for sent in j:
            context.append(sent)
context = list(set(context))

100%|██████████| 100/100 [00:00<00:00, 38714.27it/s]


In [6]:
questions = dataset['train']['question'][:100]
answer = dataset['train']['answer'][:100]

In [7]:
questions[0]

"Which magazine was started first Arthur's Magazine or First for Women?"

In [8]:
answer[0]

"Arthur's Magazine"

In [9]:
len(context)

4186

## Retrieval and Reranking

In [10]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [11]:
HF_TOKEN = "HUGGING_FACE_TOKEN"

In [12]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L12-v2", use_auth_token = HF_TOKEN)
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L12-v2", use_auth_token = HF_TOKEN)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:796: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

#### Getting embeddings of corpus and questions

In [13]:
# creating embeddings
def get_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

corpus_embeddings = [get_embedding(doc, tokenizer, model) for doc in tqdm(context)]


100%|██████████| 4186/4186 [02:16<00:00, 30.59it/s]


In [14]:
query_embedding = [get_embedding(ques, tokenizer, model) for ques in tqdm(questions)]

100%|██████████| 100/100 [00:03<00:00, 32.91it/s]


#### Creating FAISS index and searching top 10 documents for 1st query (question[0])

In [15]:
import numpy as np
# Initialize FAISS index
embedding_dim = corpus_embeddings[0].shape[0]
index = faiss.IndexFlatL2(embedding_dim)

# Add corpus embeddings to the index
corpus_embeddings_np = np.array(corpus_embeddings)
index.add(corpus_embeddings_np)

# Search for top-k nearest documents
k = 3
_, top_k_indices = index.search(np.expand_dims(query_embedding[0], axis=0), k)

# Retrieve top-k documents
top_k_documents = [context[idx] for idx in top_k_indices[0]]
print("Question:", questions[0])
print("Top-k documents:", top_k_documents)

Question: Which magazine was started first Arthur's Magazine or First for Women?
Top-k documents: ["Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.", " Founded in 1914, and first published on Women's Day, it is the first socialist women's journal, and the most politically left of the women's periodicals.", "First for Women is a woman's magazine published by Bauer Media Group in the USA."]


### Reranking

In [16]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [17]:
def sort_indices_descending(lst):
    return sorted(range(len(lst)), key=lambda i: lst[i], reverse=True)

In [18]:
def retrieve_tok_k_items(query_idx, top_k=10):
    query = [questions[query_idx]]
    q_embed = query_embedding[query_idx]
    D, I = index.search(np.expand_dims(q_embed, axis = 0), top_k)
    
    top_k_idx = I[0]
    top_k_documents = [context[idx] for idx in I[0]]

    return top_k_documents

In [19]:
def get_context_docs(query_idx):
    flat_list = [item for sublist in dataset['train']['context'][query_idx]['sentences'] for item in sublist]
    return flat_list

In [20]:
def get_reranked_docs(query_idx):
    cross_input_list = []
    top_k_documents = retrieve_tok_k_items(query_idx)
    for item in top_k_documents:
        new_list = [questions[query_idx], item]
        cross_input_list.append(new_list)
        
    cross_scores = cross_encoder.predict(cross_input_list)
    order_of_ranking = sort_indices_descending(cross_scores)
    print(order_of_ranking)
    ranked_docs = []
    is_in_orig_context = []
    flat_list = get_context_docs(query_idx)
    for j in order_of_ranking:
        ranked_docs.append(cross_input_list[j][1])
        is_in_orig_context.append(cross_input_list[j][1] in flat_list)
    
    return ranked_docs, sorted(cross_scores, reverse=True), is_in_orig_context

In [21]:
def assign_relevance_labels(reranking_scores):

    # Step 1: Normalize the reranking scores to range [0, 1]
    min_score = np.min(reranking_scores)
    max_score = np.max(reranking_scores)
    normalized_scores = (reranking_scores - min_score) / (max_score - min_score)

    # Step 2: Define thresholds for relevance bins
    n_docs = len(normalized_scores)
    sorted_indices = np.argsort(-normalized_scores)  # Sort in descending order

    top_20_percent = int(n_docs * 0.2)
    next_30_percent = int(n_docs * 0.3)

    # Step 3: Assign relevance labels based on thresholds
    relevance_labels = np.zeros(n_docs)
    relevance_labels[sorted_indices[:top_20_percent]] = 2  # Top 20% -> relevance 2
    relevance_labels[sorted_indices[top_20_percent:top_20_percent + next_30_percent]] = 1  # Next 30% -> relevance 1
    relevance_labels[sorted_indices[top_20_percent + next_30_percent:]] = 0  # Bottom 50% -> relevance 0

    return relevance_labels

"""

def assign_relevance_labels(reranking_scores):
    relevance_labels = []
    for r in reranking_scores:
        if(r>0):
            relevance_labels.append(1)
        else:
            relevance_labels.append(0)
    return relevance_labels
    
"""

'\n\ndef assign_relevance_labels(reranking_scores):\n    relevance_labels = []\n    for r in reranking_scores:\n        if(r>0):\n            relevance_labels.append(1)\n        else:\n            relevance_labels.append(0)\n    return relevance_labels\n    \n'

In [22]:
from groq import Groq

client = Groq(api_key = "GROQ_API_KEY")

In [23]:
def assign_actual_relevance_scores(df):
    df['relevance_scores_actual'] = np.nan
    for i in range(len(df)):
        completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {
                "role": "system",
                "content": "Given a context and a question, provide relevance rating to the context based on how relevant the context is in answering the question. The relevance ratings can only be one of 0 (completely irrelevant), 1(somewhat relevant), 2(completely relevant).\n\noutput only the relevance rating"
            },
            {
                "role": "user",
                "content": f"Context: {df['ranked_retrieved_docs'].iloc[i]}\n\nQuestion: {df['query'].iloc[i]}"
            }
        ],
            temperature=1,
            max_tokens=1024,
            top_p=1,
            stream=False,
            stop=None,
        )
    
        df['relevance_scores_actual'].iloc[i] = completion.choices[0].message.content
    return df

In [42]:
import pandas as pd
ranked_docs, cross_scores, is_in_orig_context = get_reranked_docs(8)
df = pd.DataFrame(ranked_docs, columns = ['ranked_retrieved_docs'])
df['ranked_retrieved_docs_score'] = cross_scores
df['is_in_orig_context'] = is_in_orig_context
df['query'] = questions[8]
relevance_labels = assign_relevance_labels(df['ranked_retrieved_docs_score'].values)
print(relevance_labels)
df['relevance_labels_reranked'] = relevance_labels
df = assign_actual_relevance_scores(df)
df

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[1, 0, 3, 4, 2, 5, 6, 7, 8, 9]
[2. 2. 1. 1. 1. 0. 0. 0. 0. 0.]


/tmp/ipykernel_31/1275284564.py:23: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['relevance_scores_actual'].iloc[i] = completion.choices[0].message.content
/tmp/ipykernel_31/1275284564.py:23: SettingWithCopyWarning: 
A value is trying to

,ranked_retrieved_docs,ranked_retrieved_docs_score,is_in_orig_context,query,relevance_labels_reranked,relevance_scores_actual
0,"Het Huis Anubis (""The House of Anubis"") is a B...",5.805280,True,"The Dutch-Belgian television series that ""Hous...",2.0,2
1,House of Anubis is a mystery television series...,5.402681,True,"The Dutch-Belgian television series that ""Hous...",2.0,1
2,Das Haus Anubis is a television program produc...,4.880110,True,"The Dutch-Belgian television series that ""Hous...",1.0,1
3,Another English remake called House of Anubis...,3.843962,True,"The Dutch-Belgian television series that ""Hous...",1.0,0
4,"This article is an episode list for ""House of ...",0.460588,True,"The Dutch-Belgian television series that ""Hous...",1.0,0
5,Ramos is known for her portrayals of Yasmin i...,-1.378016,True,"The Dutch-Belgian television series that ""Hous...",0.0,2
6,"House (also called House, M.D.) is an American...",-2.939436,False,"The Dutch-Belgian television series that ""Hous...",0.0,0
7,The series was created by Hans Bourlon and Ge...,-5.059694,True,"The Dutch-Belgian television series that ""Hous...",0.0,0
8,The show premiered on ITV on 5 January 2013 w...,-6.587852,False,"The Dutch-Belgian television series that ""Hous...",0.0,0
9,"From 29 September 2009 to 4 May 2012, the sho...",-8.522115,True,"The Dutch-Belgian television series that ""Hous...",0.0,0


In [44]:
import numpy as np
import pandas as pd


df_sorted = df.sort_values(by='relevance_labels_reranked', ascending=False)

# Calculate DCG
def dcg_at_k(relevance_scores, k):
    relevance_scores = np.asfarray(relevance_scores)[:k]
    if relevance_scores.size:
        return np.sum((2 ** relevance_scores - 1) / np.log2(np.arange(1, relevance_scores.size + 1) + 1))
    return 0.0

# Calculate IDCG (Ideal DCG) - ideal relevance is sorted in descending order
def idcg_at_k(relevance_scores, k):
    ideal_relevance = sorted(relevance_scores, reverse=True)
    return dcg_at_k(ideal_relevance, k)

# Calculate nDCG
def ndcg_at_k(actual_relevance_scores, predicted_relevance_scores, k):
    # Calculate DCG using actual relevance scores in the order of predicted relevance
    dcg = dcg_at_k(actual_relevance_scores, k)
    
    # Calculate IDCG using the actual relevance scores sorted ideally
    idcg = idcg_at_k(actual_relevance_scores, k)
    
    return dcg / idcg if idcg > 0 else 0.0

# Get the actual relevance scores after sorting by the predicted relevance
actual_relevance_scores = np.float64(df_sorted['relevance_scores_actual'])
predicted_relevance_scores = df_sorted['relevance_labels_reranked']

# Set k (you can choose a specific value or use the full list)
k = len(actual_relevance_scores)  # Or choose any k value

# Calculate nDCG
ndcg_value = ndcg_at_k(actual_relevance_scores, predicted_relevance_scores, k)
print(f"nDCG at {k}: {ndcg_value}")


nDCG at 10: 0.892861996050641


In [26]:
total_ndcg_at_10 = 0
for q in tqdm(range(len(questions))):
    ranked_docs, cross_scores, is_in_orig_context = get_reranked_docs(q)
    df = pd.DataFrame(ranked_docs, columns = ['ranked_retrieved_docs'])
    df['ranked_retrieved_docs_score'] = cross_scores
    df['is_in_orig_context'] = is_in_orig_context
    df['query'] = questions[q]
    relevance_labels = assign_relevance_labels(df['ranked_retrieved_docs_score'].values)
    df['relevance_labels_reranked'] = relevance_labels
    df = assign_actual_relevance_scores(df)
    k = 10
    df_sorted = df.sort_values(by='relevance_labels_reranked', ascending=False)
    actual_relevance_scores = df_sorted['relevance_scores_actual']
    predicted_relevance_scores = df_sorted['relevance_labels_reranked']
    ndcg_score = ndcg_at_k(actual_relevance_scores, predicted_relevance_scores, k)
    print(f"NDCG Score for Query {q} is {ndcg_score}")
    total_ndcg_at_10+=ndcg_score



  0%|          | 0/100 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[2, 0, 1, 3, 6, 4, 5, 8, 9, 7]


/tmp/ipykernel_31/1275284564.py:23: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['relevance_scores_actual'].iloc[i] = completion.choices[0].message.content
/tmp/ipykernel_31/1275284564.py:23: SettingWithCopyWarning: 
A value is trying to

NDCG Score for Query 0 is 0.9469024295259745


...

NDCG Score for Query 98 is 0.9197207891481876


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[0, 1, 2, 7, 8, 9, 5, 4, 3, 6]


/tmp/ipykernel_31/1275284564.py:23: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['relevance_scores_actual'].iloc[i] = completion.choices[0].message.content
/tmp/ipykernel_31/1275284564.py:23: SettingWithCopyWarning: 
A value is trying to

NDCG Score for Query 99 is 0.940914767159646


In [28]:
print(f"Average NDCG Score on {len(questions)} queries is: {total_ndcg_at_10/len(questions)}")

Average NDCG Score on 100 queries is: 0.8407795252785607


In [46]:
total_prec = 0
total_recall = 0
for q in tqdm(range(len(questions))):
    ranked_docs, cross_scores, is_in_orig_context = get_reranked_docs(q)
    df = pd.DataFrame(ranked_docs, columns = ['ranked_retrieved_docs'])
    df['ranked_retrieved_docs_score'] = cross_scores
    df['is_in_orig_context'] = is_in_orig_context
    df['query'] = questions[q]
    relevance_labels = assign_relevance_labels(df['ranked_retrieved_docs_score'].values)
    df['relevance_labels_reranked'] = relevance_labels
    df = assign_actual_relevance_scores(df)
    k = 10
    #df_sorted = df.sort_values(by='relevance_labels_reranked', ascending=False)
    relevant_docs = 0
    actual_relevant_docs = 0
    df['relevance_scores_actual'] = np.float64(df['relevance_scores_actual'].values)
    for i in range(k):
        if(df['relevance_labels_reranked'].iloc[i]>0 and df['relevance_scores_actual'].iloc[i]>0):
            relevant_docs+=1
        if(df['relevance_scores_actual'].iloc[i]>0):
            actual_relevant_docs+=1
    precision_at_k = relevant_docs/k
    recall_at_k = 0
    if actual_relevant_docs!=0 : 
        recall_at_k = relevant_docs/actual_relevant_docs
    
    print(f"Precision Score for Query {q} is {precision_at_k}")
    print(f"Recall Score for Query {q} is {recall_at_k}")
    total_prec+=precision_at_k
    total_recall+=recall_at_k


  0%|          | 0/100 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[2, 0, 1, 3, 6, 4, 5, 8, 9, 7]


/tmp/ipykernel_31/1275284564.py:23: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['relevance_scores_actual'].iloc[i] = completion.choices[0].message.content
/tmp/ipykernel_31/1275284564.py:23: SettingWithCopyWarning: 
A value is trying to

Precision Score for Query 0 is 0.3
Recall Score for Query 0 is 1.0


...

Precision Score for Query 98 is 0.2
Recall Score for Query 98 is 1.0


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[0, 1, 2, 7, 8, 9, 5, 4, 3, 6]


/tmp/ipykernel_31/1275284564.py:23: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['relevance_scores_actual'].iloc[i] = completion.choices[0].message.content
/tmp/ipykernel_31/1275284564.py:23: SettingWithCopyWarning: 
A value is trying to

Precision Score for Query 99 is 0.3
Recall Score for Query 99 is 1.0


In [47]:
print(f"Average Precision Score on {len(questions)} queries is: {total_prec/len(questions)}")
print(f"Average Recall Score on {len(questions)} queries is: {total_recall/len(questions)}")

Average Precision Score on 100 queries is: 0.2889999999999999
Average Recall Score on 100 queries is: 0.7707380952380953
